In [ ]:
#!/usr/bin/env python3
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def load_prices(prices_file="prices.txt"):
    """
    Load (timesteps × instruments) price matrix.
    Assumes whitespace-delimited, oldest row first.
    """
    return np.loadtxt(Path(prices_file), delimiter=None)

def find_pips(prices: np.ndarray, threshold: float):
    """
    Zig-zag pivot finder, 100% causal:
    uses only prices up to the current bar.

    Returns a list of (idx, price) pairs where each
    pivot is detected exactly at the bar it occurs.
    """
    n = len(prices)
    pips = [(0, prices[0])]
    last_extreme = prices[0]
    direction = None

    for i in range(1, n):
        move = (prices[i] - last_extreme) / last_extreme

        if direction is None:
            if abs(move) >= threshold:
                direction     = 'up' if move > 0 else 'down'
                last_extreme  = prices[i]
                pips.append((i, prices[i]))
        else:
            # extend the current swing
            if direction=='up' and prices[i] > last_extreme:
                last_extreme = prices[i]
            elif direction=='down' and prices[i] < last_extreme:
                last_extreme = prices[i]

            # check for a reversal
            rev = ((last_extreme - prices[i]) / last_extreme
                   if direction=='up'
                   else (prices[i] - last_extreme) / last_extreme)
            if rev >= threshold:
                pips.append((i, last_extreme))
                direction    = 'down' if direction=='up' else 'up'
                last_extreme = prices[i]

    # always include the final bar as a pivot
    if pips[-1][0] != n-1:
        pips.append((n-1, prices[-1]))

    return pips

def causal_signal(series: np.ndarray, threshold: float):
    """
    One-pass, fully causal up/down signal:
      +1 = long, -1 = short, 0 = flat until first swing confirms.
    """
    n = len(series)
    pos = np.zeros(n, dtype=int)
    last_extreme = series[0]
    direction = None

    for i in range(1, n):
        move = (series[i] - last_extreme) / last_extreme

        if direction is None:
            if abs(move) >= threshold:
                direction    = 'up' if move > 0 else 'down'
                last_extreme = series[i]
        else:
            if direction=='up':
                if series[i] > last_extreme:
                    last_extreme = series[i]
                else:
                    rev = (last_extreme - series[i]) / last_extreme
                    if rev >= threshold:
                        direction    = 'down'
                        last_extreme = series[i]
            else:  # direction == 'down'
                if series[i] < last_extreme:
                    last_extreme = series[i]
                else:
                    rev = (series[i] - last_extreme) / last_extreme
                    if rev >= threshold:
                        direction    = 'up'
                        last_extreme = series[i]

        pos[i] = 1 if direction=='up' else (-1 if direction=='down' else 0)

    return pos

if __name__ == "__main__":
    # ───── Load Data ─────
    try:
        prices = load_prices("prices-2024.txt")
        print("Loaded prices.txt")
    except FileNotFoundError:
        np.random.seed(0)
        prices = np.cumsum(np.random.randn(1000, 50), axis=0) + 100.0
        print("Using synthetic data (1000×50)")

    # ───── Parameters ─────
    instr     = 15      # instrument to plot
    t1, t2    = 600, 700
    threshold = 0.01   # 1% pivot threshold

    # ───── Slice & Compute ─────
    series = prices[t1:t2+1, instr]
    times  = np.arange(t1, t2+1)

    pos    = causal_signal(series, threshold)
    pivots = find_pips(series, threshold)

    # ───── Shift PIP markers left by one bar (for display only) ─────
    # This does NOT affect the trading logic!
    shifted = []
    for idx, price in pivots:
        if idx > 0:
            shifted.append((idx-1, series[idx-1]))
    # if the final pivot was at the last bar, make sure it shows up:
    if shifted and shifted[-1][0] != len(series)-1:
        shifted.append((len(series)-1, series[-1]))
    if not shifted:
        shifted = [(0, series[0])]

    shift_idxs, shift_vals = zip(*shifted)

    # ───── Plot ─────
    fig, ax1 = plt.subplots(figsize=(12,5))

    # shade price-underlay
    ax1.fill_between(times, series, series.min(),
                     where=pos>0, facecolor='green', alpha=0.2,
                     step='post')
    ax1.fill_between(times, series, series.min(),
                     where=pos<0, facecolor='red',   alpha=0.2,
                     step='post')

    # price line
    ax1.plot(times, series, color='black', linewidth=1,
             label=f"Inst {instr}")

    # shifted pivot markers
    ax1.scatter(times[list(shift_idxs)], shift_vals,
                color='blue', s=50, zorder=3,
                label="PIPs (shifted left 1 bar)")

    ax1.set_xlabel("Timestep")
    ax1.set_ylabel("Price")
    ax1.set_title("Fully Causal PIP-Based Up/Down Indicator")

    # overlay live trading position
    ax2 = ax1.twinx()
    ax2.step(times, pos, where='post',
             color='gray', linewidth=0.01,
             label="Position (+1 long / -1 short)")
    ax2.set_ylim(-1.2, 1.2)
    ax2.set_ylabel("Position")

    # combine legends
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2,
               loc='upper left', fontsize='small')

    fig.tight_layout()
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
gridsearch_pip_threshold.py
───────────────────────────
For each instrument, sweep pivot‐thresholds from 0.005 to 0.500
(step 0.0005) and pick the one that maximises **net PnL** after a
commission of 0.0005 per dollar traded.

Trading model
-------------
• Position = ±POSITION_SIZE shares when the causal PIP direction is
  `up` / `down`; 0 when flat.
• PnL for bar *t* is  
      pos[t-1] · (price[t] − price[t-1])
  (Mark-to-market at next close.)
• Commission on a position change is  
      COMM_RATE · |Δpos[t]| · price[t]

The code prints a per-instrument summary and an overall total.
"""
from pathlib import Path
from typing import Tuple

import numpy as np

# ───────────────────────────────── CONSTANTS ──────────────────────────────
PRICE_FILE      = "prices.txt"          # whitespace-delimited
THRESH_MIN      = 0.005
THRESH_MAX      = 0.500
THRESH_STEP     = 0.0005
POSITION_SIZE   = 10_000                # shares per instrument
COMM_RATE       = 0.0005                # per dollar traded

# ──────────────────────────── helpers ─────────────────────────────────────
def load_prices(fname: str = PRICE_FILE) -> np.ndarray:
    """
    Return a (T × nInst) price matrix.
    """
    f = Path(fname)
    if not f.exists():
        raise FileNotFoundError(f"{fname!s} not found")
    px = np.loadtxt(f, delimiter=None)
    # shape guard: rows=time, cols=instrument
    if px.ndim != 2:
        raise ValueError("Expecting 2-D price matrix (T × nInst)")
    return px


def causal_direction(series: np.ndarray, thr: float) -> np.ndarray:
    """
    Vector of +1/0/-1 directions for a single instrument given `thr`.
    (Pure Python loop is fine – T is usually O(10³–10⁴).)
    """
    n = len(series)
    out = np.zeros(n, dtype=np.int8)

    last_extreme = series[0]
    direction    = None                   # 'up' | 'down' | None

    for i in range(1, n):
        move = (series[i] - last_extreme) / last_extreme

        if direction is None:
            if abs(move) >= thr:
                direction    = 'up' if move > 0 else 'down'
                last_extreme = series[i]
        else:
            if direction == 'up':
                if series[i] > last_extreme:          # extend rally
                    last_extreme = series[i]
                elif (last_extreme - series[i]) / last_extreme >= thr:
                    direction    = 'down'             # reversal
                    last_extreme = series[i]
            else:                                     # direction == 'down'
                if series[i] < last_extreme:          # extend decline
                    last_extreme = series[i]
                elif (series[i] - last_extreme) / last_extreme >= thr:
                    direction    = 'up'               # reversal
                    last_extreme = series[i]

        out[i] = 1 if direction == 'up' else (-1 if direction == 'down' else 0)
    return out


# ────────────────── simulate_pnl (patched) ──────────────────
def simulate_pnl(series: np.ndarray,
                 thr: float,
                 pos_size: int = POSITION_SIZE,
                 comm_rate: float = COMM_RATE
                 ) -> Tuple[float, float, float, int]:

    # 1) generate ±1/0 signal
    dir_vec = causal_direction(series, thr)

    # --- PATCH ①: promote to 32-bit before scaling ------------------------
    pos = dir_vec.astype(np.int32) * pos_size          # shares held
    # ----------------------------------------------------------------------

    # 2) increments
    price_diff = np.diff(series, prepend=series[0])

    # --- PATCH ②: cast to float64 for safe arithmetic ---------------------
    pnl = pos[:-1].astype(np.float64) * price_diff[1:]   # bar-to-bar PnL
    # ----------------------------------------------------------------------

    delta_pos   = np.diff(pos, prepend=0)

    # --- PATCH ③: commission also in float64 ------------------------------
    commission  = comm_rate * np.abs(delta_pos).astype(np.float64) * series
    # ----------------------------------------------------------------------

    gross_pnl   = float(pnl.sum())
    total_comm  = float(commission.sum())
    net_pnl     = gross_pnl - total_comm
    n_trades    = int((delta_pos != 0).sum())

    return net_pnl, gross_pnl, total_comm, n_trades



def best_threshold(series: np.ndarray) -> Tuple[float, float, float, float, int]:
    """
    Grid-search thresholds and return:
      best_thr, best_net_pnl, gross_pnl, commission, n_trades
    """
    best_thr   = THRESH_MIN
    best_stats = (-np.inf, 0.0, 0.0, 0)   # net, gross, comm, trades

    thr = THRESH_MIN
    while thr <= THRESH_MAX + 1e-12:      # float guard
        net, gross, comm, n_tr = simulate_pnl(series, thr)
        if net > best_stats[0]:
            best_thr   = thr
            best_stats = (net, gross, comm, n_tr)
        thr += THRESH_STEP
    return (best_thr, *best_stats)


# ─────────────────────────────── main ─────────────────────────────────────
def main() -> None:
    px = load_prices()                    # (T × nInst)
    T, nInst = px.shape
    print(f"Loaded price matrix: {T:,d} timesteps × {nInst} instruments\n")

    hdr = ("Instr", "BestThr", "NetPnL", "Gross", "Comm", "Trades")
    print(f"{hdr[0]:>5}  {hdr[1]:>7}  {hdr[2]:>12}  {hdr[3]:>12}  "
          f"{hdr[4]:>10}  {hdr[5]:>6}")
    print("-"*60)

    total_net = total_gross = total_comm = 0.0
    total_trades = 0

    for i in range(nInst):
        best_thr, net, gross, comm, n_tr = best_threshold(px[:, i])

        total_net    += net
        total_gross  += gross
        total_comm   += comm
        total_trades += n_tr

        print(f"{i:5d}  {best_thr:7.4f}  {net:12,.2f}  {gross:12,.2f}  "
              f"{comm:10,.2f}  {n_tr:6d}")

    print("-"*60)
    print(f"Total net PnL : {total_net:,.2f}")
    print(f"   Gross PnL  : {total_gross:,.2f}")
    print(f"   Commission : {total_comm:,.2f}")
    print(f"   Trades     : {total_trades:,d}")


if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
gridsearch_pip_threshold_compare.py
───────────────────────────────────
For each instrument:

1.  Grid-search pivot thresholds from **0.005 → 0.500**
   (step 0.0005) on the **full history**.
2.  Repeat the same search using only the **first 500 bars**.
3.  Report, side-by-side, the optimal threshold, net PnL, and the
   standard deviation (σ) of per-bar **net** PnL for both windows.

Trading assumptions
-------------------
• Position = ±POSITION_SIZE shares when the causal PIP direction is
  `up` / `down`; 0 when flat.
• Net PnL per bar *t* = position[t-1] · (price[t]−price[t-1]) −
  commission on any change executed at bar *t*.
• Commission per change = COMM_RATE · |Δposition| · price[t].
"""
from pathlib import Path
from typing   import Tuple

import numpy as np

# ───────────────────────────────── CONSTANTS ──────────────────────────────
PRICE_FILE      = "prices.txt"          # whitespace-delimited
THRESH_MIN      = 0.000
THRESH_MAX      = 0.700
THRESH_STEP     = 0.0005
POSITION_SIZE   = 10_000                # shares per instrument
COMM_RATE       = 0.0005                # per dollar traded
FIRST_N_BARS    = 700                   # early-window length

# ──────────────────────────── data loader ─────────────────────────────────
def load_prices(fname: str = PRICE_FILE) -> np.ndarray:
    f = Path(fname)
    if not f.exists():
        raise FileNotFoundError(f"{fname!s} not found")
    px = np.loadtxt(f, delimiter=None)
    if px.ndim != 2:
        raise ValueError("Expecting 2-D price matrix (T × nInst)")
    return px


# ─────────────────────── causal PIP direction ────────────────────────────
def causal_direction(series: np.ndarray, thr: float) -> np.ndarray:
    """
    Vector of +1 / 0 / -1 directions for one instrument.
    """
    n   = len(series)
    out = np.zeros(n, dtype=np.int8)

    last_extreme = series[0]
    direction    = None            # 'up' | 'down' | None

    for i in range(1, n):
        move = (series[i] - last_extreme) / last_extreme

        if direction is None:
            if abs(move) >= thr:
                direction    = 'up' if move > 0 else 'down'
                last_extreme = series[i]
        else:
            if direction == 'up':
                if series[i] > last_extreme:
                    last_extreme = series[i]
                elif (last_extreme - series[i]) / last_extreme >= thr:
                    direction    = 'down'
                    last_extreme = series[i]
            else:                                  # direction == 'down'
                if series[i] < last_extreme:
                    last_extreme = series[i]
                elif (series[i] - last_extreme) / last_extreme >= thr:
                    direction    = 'up'
                    last_extreme = series[i]

        out[i] = 1 if direction == 'up' else (-1 if direction == 'down' else 0)
    return out


# ───────────────────── simulate PnL for one thr ──────────────────────────
def simulate_pnl(series: np.ndarray,
                 thr: float,
                 pos_size: int = POSITION_SIZE,
                 comm_rate: float = COMM_RATE
                 ) -> Tuple[float, float]:
    """
    Return (net_pnl_total, std_net_pnl_per_bar)
    """
    # 1) ±1/0 signal → shares
    dir_vec = causal_direction(series, thr)
    pos     = dir_vec.astype(np.int32) * pos_size

    # 2) per-bar PnL & commission
    price_diff  = np.diff(series, prepend=series[0])
    gross_pnl   = pos[:-1].astype(np.float64) * price_diff[1:]

    delta_pos   = np.diff(pos, prepend=0)
    commission  = comm_rate * np.abs(delta_pos).astype(np.float64) * series

    net_pnl_vec = gross_pnl - commission[1:]     # align to bar t
    net_total   = float(net_pnl_vec.sum())
    std_net     = float(np.std(net_pnl_vec, ddof=0))

    return net_total, std_net


# ─────────────────── grid-search best threshold ──────────────────────────
def best_threshold(series: np.ndarray) -> Tuple[float, float, float]:
    """
    Return (best_thr, best_net_pnl, std_net_pnl)
    """
    best_thr   = THRESH_MIN
    best_net   = -np.inf
    best_std   = 0.0

    thr = THRESH_MIN
    while thr <= THRESH_MAX + 1e-12:
        net, std = simulate_pnl(series, thr)
        if net > best_net:
            best_thr, best_net, best_std = thr, net, std
        thr += THRESH_STEP
    return best_thr, best_net, best_std


# ───────────────────────────────── main ───────────────────────────────────
def main() -> None:
    px = load_prices()                      # (T × nInst)
    T, nInst = px.shape
    if T < FIRST_N_BARS:
        raise ValueError(f"Need at least {FIRST_N_BARS} bars; only {T} found")

    print(f"Loaded price matrix: {T:,d} timesteps × {nInst} instruments\n")

    hdr = ("Inst", "Thr-Full", "Thr-500", "Δthr",
           "Net-Full", "σ-Full", "Net-500", "σ-500")
    print(f"{hdr[0]:>5}  {hdr[1]:>7}  {hdr[2]:>7}  {hdr[3]:>7}  "
          f"{hdr[4]:>12}  {hdr[5]:>8}  {hdr[6]:>12}  {hdr[7]:>8}")
    print("-"*85)

    tot_full = tot_early = 0.0

    for i in range(nInst):
        series_full  = px[:, i]
        series_early = px[:FIRST_N_BARS, i]

        thr_full,  net_full,  std_full  = best_threshold(series_full)
        thr_early, net_early, std_early = best_threshold(series_early)

        d_thr = thr_early - thr_full

        tot_full  += net_full
        tot_early += net_early

        print(f"{i:5d}  "
              f"{thr_full:7.4f}  {thr_early:7.4f}  {d_thr:7.4f}  "
              f"{net_full:12,.2f}  {std_full:8.2f}  "
              f"{net_early:12,.2f}  {std_early:8.2f}")

    print("-"*85)
    print(f"Σ Net PnL (full) : {tot_full:,.2f}")
    print(f"Σ Net PnL (first {FIRST_N_BARS}) : {tot_early:,.2f}")


if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
dynamic_threshold_gridsearch.py
────────────────────────────────
Dynamically re-calibrates each instrument’s causal-PIP threshold:

    • Window length  N  ∈ N_GRID      (number of bars to train on)
    • Re-calibration step W ∈ W_GRID  (bars between re-fits)

For every (N, W) in the outer grid, the inner routine re-optimises the
threshold on the trailing N-bar window (grid THR_GRID) and trades with
±POSITION_SIZE shares until the next calibration point.

Outputs, per instrument:

    Inst   Best-N   Best-W   Dyn-PnL   StaticThr   Static-PnL

with portfolio totals at the end, all after a 0.0005 commission.
"""
from __future__ import annotations
from pathlib import Path
from typing  import Tuple, Sequence

import numpy as np

# ═════════════════════════ CONSTANTS ═════════════════════════════════════
PRICE_FILE    = "prices.txt"          # whitespace-delimited T×50
POSITION_SIZE = 10_000               # ±shares on a signal
COMM_RATE     = 0.0005               # 5 bp per dollar traded

# Threshold grid (inner)
THRESH_MIN  = 0.000
THRESH_MAX  = 0.700
THRESH_STEP = 0.0005
THR_GRID    = np.arange(THRESH_MIN, THRESH_MAX + THRESH_STEP, THRESH_STEP)

# Outer (N, W) parameter grids
N_GRID: Sequence[int] = [100, 200, 300, 400, 500]
W_GRID: Sequence[int] = [ 10,  25,  50, 100]

# ════════════════════════ DATA LOADER ════════════════════════════════════
def load_prices(fname: str = PRICE_FILE) -> np.ndarray:
    f = Path(fname)
    if not f.exists():
        raise FileNotFoundError(f"{fname!s} not found")
    px = np.loadtxt(f, delimiter=None)
    if px.ndim != 2:
        raise ValueError("Expecting 2-D price matrix (T × nInst)")
    return px


# ════════════════  Causal PIP Direction (int8 vector) ════════════════════
def causal_direction(series: np.ndarray, thr: float) -> np.ndarray:
    """
    Returns a vector of +1/0/-1 for one instrument.
    """
    n   = len(series)
    out = np.zeros(n, dtype=np.int8)

    last_extreme = series[0]
    direction    = None  # 'up' | 'down' | None

    for i in range(1, n):
        move = (series[i] - last_extreme) / last_extreme

        if direction is None:
            if abs(move) >= thr:
                direction    = 'up' if move > 0 else 'down'
                last_extreme = series[i]
        else:
            if direction == 'up':
                if series[i] > last_extreme:
                    last_extreme = series[i]
                elif (last_extreme - series[i]) / last_extreme >= thr:
                    direction, last_extreme = 'down', series[i]
            else:  # direction == 'down'
                if series[i] < last_extreme:
                    last_extreme = series[i]
                elif (series[i] - last_extreme) / last_extreme >= thr:
                    direction, last_extreme = 'up', series[i]

        out[i] = 1 if direction == 'up' else (-1 if direction == 'down' else 0)
    return out


# ═══════════════  Static-Threshold PnL (baseline) ════════════════════════
def pnl_static(series: np.ndarray, thr: float) -> float:
    dir_vec = causal_direction(series, thr)
    pos     = dir_vec.astype(np.int32) * POSITION_SIZE   # ← 32-bit now

    price_diff = np.diff(series, prepend=series[0])
    gross_pnl  = pos[:-1].astype(np.float64) * price_diff[1:]

    delta_pos  = np.diff(pos, prepend=0)
    commission = COMM_RATE * np.abs(delta_pos).astype(np.float64) * series

    return float(gross_pnl.sum() - commission[1:].sum())


def best_static_threshold(series: np.ndarray) -> Tuple[float, float]:
    best_thr = THR_GRID[0]
    best_pnl = -np.inf
    for thr in THR_GRID:
        pnl = pnl_static(series, thr)
        if pnl > best_pnl:
            best_thr, best_pnl = thr, pnl
    return best_thr, best_pnl


# ═════════════ Dynamic PnL with (N,W) Re-calibration ═════════════════════
def pnl_dynamic(series: np.ndarray, N: int, W: int) -> float:
    """
    Re-fit the threshold on the last N bars every W bars.
    """
    T   = len(series)
    pos = np.zeros(T, dtype=np.int32)  # 32-bit container

    if N >= T:
        return 0.0

    current_thr: float | None = None

    for t in range(T):
        if t < N:
            continue

        # re-fit at t == N and then every W bars
        if (t == N) or ((t - N) % W == 0):
            window = series[t - N:t]
            current_thr, _ = best_static_threshold(window)

        # latest direction from causal signal up to bar t
        dir_now = int(causal_direction(series[:t + 1], current_thr)[-1])
        pos[t]  = dir_now * POSITION_SIZE

    price_diff = np.diff(series, prepend=series[0])
    gross_pnl  = pos[:-1].astype(np.float64) * price_diff[1:]

    delta_pos  = np.diff(pos, prepend=0)
    commission = COMM_RATE * np.abs(delta_pos).astype(np.float64) * series

    return float(gross_pnl.sum() - commission[1:].sum())


def best_dynamic_params(series: np.ndarray) -> Tuple[int, int, float]:
    """
    Grid-search N×W for this instrument.
    """
    best_pnl = -np.inf
    best_N = best_W = None
    for N in N_GRID:
        if N >= len(series):
            continue
        for W in W_GRID:
            if W <= 0:
                continue
            pnl = pnl_dynamic(series, N, W)
            if pnl > best_pnl:
                best_pnl, best_N, best_W = pnl, N, W
    return best_N, best_W, best_pnl


# ═══════════════════════════ MAIN ════════════════════════════════════════
def main() -> None:
    prices = load_prices()
    T, nInst = prices.shape
    print(f"Loaded prices: {T:,d} bars × {nInst} instruments\n")

    hdr = ("Inst", "Best-N", "Best-W", "Dyn-PnL", "StaticThr", "Static-PnL")
    print(f"{hdr[0]:>5}  {hdr[1]:>7}  {hdr[2]:>7}  {hdr[3]:>12}  "
          f"{hdr[4]:>10}  {hdr[5]:>12}")
    print("─" * 70)

    tot_dyn = tot_static = 0.0

    for i in range(nInst):
        series = prices[:, i]

        # dynamic grid-search
        N_opt, W_opt, pnl_dyn = best_dynamic_params(series)
        tot_dyn += pnl_dyn

        # static best threshold
        thr_static, pnl_static_best = best_static_threshold(series)
        tot_static += pnl_static_best

        print(f"{i:5d}  "
              f"{N_opt:7d}  {W_opt:7d}  {pnl_dyn:12,.2f}  "
              f"{thr_static:10.4f}  {pnl_static_best:12,.2f}")

    print("─" * 70)
    print(f"Σ Net PnL (dynamic): {tot_dyn:,.2f}")
    print(f"Σ Net PnL (static) : {tot_static:,.2f}")


if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
dynamic_threshold_gridsearch_fast.py
────────────────────────────────────
Same logic as before — dynamically re-calibrate a causal-PIP threshold
every W bars using a trailing N-bar window — but accelerated with
Numba + multiprocessing.

Outputs (per instrument):
    Inst   Best-N   Best-W   Dyn-PnL   StaticThr   Static-PnL
"""
from __future__ import annotations
from pathlib import Path
from typing  import Tuple, Sequence, List

import numpy as np
from numba import njit, prange                       # JIT compiler
from joblib import Parallel, delayed                 # multiprocess map

# ═════════════════════════ CONSTANTS ═════════════════════════════════════
PRICE_FILE    = "prices.txt"      # whitespace-delimited T×50
POSITION_SIZE = 10_000            # ±shares when in-market
COMM_RATE     = 0.0005            # 5 bp per dollar traded

# Threshold grid (inner search)
THRESH_MIN  = 0.0000
THRESH_MAX  = 0.7000
THRESH_STEP = 0.0005
THR_GRID    = np.arange(THRESH_MIN, THRESH_MAX + THRESH_STEP, THRESH_STEP,
                        dtype=np.float64)            # JIT-friendly

# Outer (N, W) grids — tweak for speed/coverage
N_GRID: Sequence[int] = [100, 200, 300, 400, 500]
W_GRID: Sequence[int] = [10, 25, 50, 100]

# ════════════════════════ DATA LOADER ════════════════════════════════════
def load_prices(fname: str = PRICE_FILE) -> np.ndarray:
    px = np.loadtxt(Path(fname), delimiter=None)
    if px.ndim != 2:
        raise ValueError("Expecting 2-D price matrix (T × nInst)")
    return px


# ═════════════════════════   NUMBA CORE   ════════════════════════════════
@njit(cache=True, fastmath=True)
def _causal_dir(series: np.ndarray, thr: float) -> np.ndarray:
    """
    Vector of +1/0/-1 (int8) for one instrument & threshold.
    """
    n    = series.size
    out  = np.zeros(n, np.int8)
    le   = series[0]           # last extreme
    dirc = 0                   # 0 none, 1 up, -1 down

    for i in range(1, n):
        move = (series[i] - le) / le

        if dirc == 0:
            if abs(move) >= thr:
                dirc = 1 if move > 0 else -1
                le   = series[i]
        elif dirc == 1:
            if series[i] > le:
                le = series[i]
            elif (le - series[i]) / le >= thr:
                dirc = -1
                le   = series[i]
        else:                  # dirc == -1
            if series[i] < le:
                le = series[i]
            elif (series[i] - le) / le >= thr:
                dirc = 1
                le   = series[i]

        out[i] = dirc
    return out


@njit(cache=True, fastmath=True)
def _pnl_static(series: np.ndarray, thr: float,
                pos_size: int, comm_rate: float) -> float:
    dir_vec = _causal_dir(series, thr)
    n       = series.size
    pos     = dir_vec.astype(np.int32) * pos_size

    pnl = 0.0
    comm = 0.0
    for i in range(1, n):
        pnl  += pos[i-1] * (series[i] - series[i-1])
        comm += comm_rate * abs(pos[i] - pos[i-1]) * series[i]
    return pnl - comm


@njit(cache=True, fastmath=True)
def _best_thr(window: np.ndarray, thr_grid: np.ndarray,
              pos_size: int, comm_rate: float) -> float:
    best_thr = thr_grid[0]
    best_pnl = -1e30
    for thr in thr_grid:
        pnl = _pnl_static(window, thr, pos_size, comm_rate)
        if pnl > best_pnl:
            best_pnl = pnl
            best_thr = thr
    return best_thr


@njit(cache=True, fastmath=True)
def _pnl_dynamic(series: np.ndarray,
                 N: int, W: int,
                 thr_grid: np.ndarray,
                 pos_size: int, comm_rate: float) -> float:
    """
    Fast JIT version of rolling-recalibration PnL.
    """
    T   = series.size
    pos = np.zeros(T, np.int32)

    if N >= T:
        return 0.0

    current_thr = _best_thr(series[:N], thr_grid, pos_size, comm_rate)

    # incremental state for causal signal
    dirc = 0
    last_extreme = series[0]

    for t in range(1, T):
        px = series[t]
        move = (px - last_extreme) / last_extreme

        # ---- causal signal update (O(1)) ------------------------------
        if dirc == 0:
            if abs(move) >= current_thr:
                dirc        = 1 if move > 0 else -1
                last_extreme = px
        elif dirc == 1:
            if px > last_extreme:
                last_extreme = px
            elif (last_extreme - px) / last_extreme >= current_thr:
                dirc        = -1
                last_extreme = px
        else:                                # dirc == -1
            if px < last_extreme:
                last_extreme = px
            elif (px - last_extreme) / last_extreme >= current_thr:
                dirc        = 1
                last_extreme = px
        # ----------------------------------------------------------------

        pos[t] = dirc * pos_size

        # re-fit after updating bar t so we *trade* using old thr for bar t
        if t >= N and ((t - N) % W == 0) and t != T-1:
            current_thr = _best_thr(series[t - N + 1:t + 1],
                                    thr_grid, pos_size, comm_rate)
            # reset causal state to be consistent with new thr
            dirc_vec = _causal_dir(series[:t+1], current_thr)
            dirc      = dirc_vec[-1]
            # recompute last_extreme consistent with dirc
            last_extreme = series[t]
            if dirc != 0:
                # walk backwards until pivot
                for back in range(t, -1, -1):
                    last_extreme = series[back]
                    if dirc == 1 and series[back] < last_extreme: break
                    if dirc == -1 and series[back] > last_extreme: break

    # ---- compute net PnL ----------------------------------------------
    pnl = 0.0
    comm = 0.0
    for i in range(1, T):
        pnl  += pos[i-1] * (series[i] - series[i-1])
        comm += comm_rate * abs(pos[i]-pos[i-1]) * series[i]
    return pnl - comm


# ═════════════  Wrapper for one instrument (allows joblib) ═══════════════
def evaluate_instrument(idx: int, series: np.ndarray) -> Tuple[int,int,float,
                                                               float,float]:
    """
    Returns (best_N, best_W, pnl_dyn, best_static_thr, pnl_static)
    """
    # static baseline
    best_thr, best_static_pnl = -1.0, -1e30
    for thr in THR_GRID:
        pnl = _pnl_static(series, thr, POSITION_SIZE, COMM_RATE)
        if pnl > best_static_pnl:
            best_thr, best_static_pnl = thr, pnl

    # dynamic grid
    best_N = best_W = 0
    best_dyn = -1e30
    for N in N_GRID:
        if N >= series.size:
            continue
        for W in W_GRID:
            dyn = _pnl_dynamic(series, N, W, THR_GRID,
                               POSITION_SIZE, COMM_RATE)
            if dyn > best_dyn:
                best_dyn = dyn
                best_N   = N
                best_W   = W

    return best_N, best_W, best_dyn, best_thr, best_static_pnl


# ═══════════════════════════ MAIN ════════════════════════════════════════
def main() -> None:
    prices = load_prices()
    T, nInst = prices.shape
    print(f"Loaded prices: {T:,d} bars × {nInst} instruments\n")

    # Parallel evaluation across CPU cores
    results: List[Tuple[int,int,float,float,float]] = Parallel(
        n_jobs=-1, backend="loky", verbose=0)(
        delayed(evaluate_instrument)(i, prices[:, i]) for i in range(nInst)
    )

    hdr = ("Inst", "Best-N", "Best-W", "Dyn-PnL", "StaticThr", "Static-PnL")
    print(f"{hdr[0]:>5}  {hdr[1]:>7}  {hdr[2]:>7}  {hdr[3]:>12}  "
          f"{hdr[4]:>10}  {hdr[5]:>12}")
    print("─"*70)

    tot_dyn = tot_static = 0.0
    for i, (N_opt, W_opt, pnl_dyn, thr_stat, pnl_stat) in enumerate(results):
        tot_dyn    += pnl_dyn
        tot_static += pnl_stat
        print(f"{i:5d}  {N_opt:7d}  {W_opt:7d}  {pnl_dyn:12,.2f}  "
              f"{thr_stat:10.4f}  {pnl_stat:12,.2f}")

    print("─"*70)
    print(f"Σ Net PnL (dynamic): {tot_dyn:,.2f}")
    print(f"Σ Net PnL (static) : {tot_static:,.2f}")


if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
dynamic_threshold_gridsearch_fast.py  (causal & accelerated)
─────────────────────────────────────────────────────────────
* Per instrument, evaluate every (N, W) in N_GRID × W_GRID:
    – Fit the best pivot-threshold on the last N bars.
    – Trade the **next W bars** with that fixed threshold
      (position = ±POSITION_SIZE shares via causal PIP signal).
    – Re-fit and repeat until the end of the series.
* Compare that dynamic PnL with the best one-shot static threshold.
* Uses Numba JIT + joblib multiprocessing for speed.
"""
from __future__ import annotations
from pathlib   import Path
from typing    import Sequence, Tuple, List

import numpy as np
from numba import njit, prange
from joblib import Parallel, delayed

# ════════════════════════════ CONFIG ═════════════════════════════════════
PRICE_FILE    = "prices.txt"        # whitespace-delimited (T × 50)
POSITION_SIZE = 10_000              # shares long / short
COMM_RATE     = 0.0005              # 5 bp per dollar traded

# Inner threshold grid
THR_MIN   = 0.0
THR_MAX   = 0.700
THR_STEP  = 0.0005
THR_GRID  = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=np.float64)

# Outer (N, W) calibration grids
N_GRID: Sequence[int] = [100, 200, 300, 400, 500]
W_GRID: Sequence[int] = [10, 25, 50, 100]

# ═══════════════════════ HELPERS / IO ════════════════════════════════════
def load_prices(fname: str = PRICE_FILE) -> np.ndarray:
    arr = np.loadtxt(Path(fname), delimiter=None)
    if arr.ndim != 2:
        raise ValueError("prices.txt must be 2-D (T × nInst)")
    return arr

# ═══════════════════════ CORE NUMBA JITs ═════════════════════════════════
@njit(cache=True, fastmath=True)
def _causal_dir(series: np.ndarray, thr: float) -> np.ndarray:
    """Return int8 vector {-1,0,+1} for one instrument."""
    n   = series.size
    out = np.zeros(n, np.int8)
    le  = series[0]      # last extreme
    d   = 0              # direction 0/±1

    for i in range(1, n):
        move = (series[i] - le) / le
        if d == 0:
            if abs(move) >= thr:
                d  = 1 if move > 0 else -1
                le = series[i]
        elif d == 1:
            if series[i] > le:
                le = series[i]
            elif (le - series[i]) / le >= thr:
                d  = -1
                le = series[i]
        else:            # d == -1
            if series[i] < le:
                le = series[i]
            elif (series[i] - le) / le >= thr:
                d  = 1
                le = series[i]
        out[i] = d
    return out


@njit(cache=True, fastmath=True)
def _pnl_static(series: np.ndarray,
                thr: float,
                pos_size: int,
                comm_rate: float) -> float:
    dir_vec = _causal_dir(series, thr)
    pos     = dir_vec.astype(np.int32) * pos_size

    pnl = 0.0
    comm = 0.0
    for i in range(1, series.size):
        pnl  += pos[i-1] * (series[i] - series[i-1])
        comm += comm_rate * abs(pos[i] - pos[i-1]) * series[i]
    return pnl - comm


@njit(cache=True, fastmath=True)
def _best_thr(window: np.ndarray,
              thr_grid: np.ndarray,
              pos_size: int,
              comm_rate: float) -> float:
    """Return threshold with max net-PnL on *window*."""
    best_thr = thr_grid[0]
    best_pnl = -1e30
    for thr in thr_grid:
        pnl = _pnl_static(window, thr, pos_size, comm_rate)
        if pnl > best_pnl:
            best_pnl = pnl
            best_thr = thr
    return best_thr


@njit(cache=True, fastmath=True)
def _pnl_dynamic(series: np.ndarray,
                 N: int, W: int,
                 thr_grid: np.ndarray,
                 pos_size: int,
                 comm_rate: float) -> float:
    """
    True walk-forward: fit on [t-N, t-1], trade bars t … t+W-1,
    then slide window W bars forward.
    """
    T   = series.size
    pos = np.zeros(T, np.int32)

    if N >= T:
        return 0.0

    seg_start = N
    while seg_start < T:
        # fit on trailing N bars *ending at seg_start-1*
        thr = _best_thr(series[seg_start-N: seg_start],
                        thr_grid, pos_size, comm_rate)

        seg_end = min(seg_start + W, T)
        # trade this segment with fixed thr
        d_vec = _causal_dir(series[:seg_end], thr)
        for t in range(seg_start, seg_end):
            pos[t] = d_vec[t] * pos_size

        seg_start += W

    # ------------ compute net PnL -------------------------------------
    pnl = comm = 0.0
    for i in range(1, T):
        pnl  += pos[i-1] * (series[i] - series[i-1])
        comm += comm_rate * abs(pos[i] - pos[i-1]) * series[i]
    return pnl - comm

# ═════════════ wrappers for multiprocessing / evaluation ════════════════
def eval_instrument(idx: int,
                    series: np.ndarray
                    ) -> Tuple[int,int,float,float,float]:
    """Return (best_N, best_W, dyn_PnL, static_thr, static_PnL)."""
    # static best threshold
    best_thr, best_static = -1.0, -1e30
    for thr in THR_GRID:
        pnl = _pnl_static(series, thr, POSITION_SIZE, COMM_RATE)
        if pnl > best_static:
            best_thr, best_static = thr, pnl

    # dynamic grid
    best_N = best_W = 0
    best_dyn = -1e30
    for N in N_GRID:
        if N >= series.size:
            continue
        for W in W_GRID:
            pnl = _pnl_dynamic(series, N, W, THR_GRID,
                               POSITION_SIZE, COMM_RATE)
            if pnl > best_dyn:
                best_dyn, best_N, best_W = pnl, N, W

    return best_N, best_W, best_dyn, best_thr, best_static


# ═══════════════════════════ MAIN ════════════════════════════════════════
def main() -> None:
    prices = load_prices()
    T, nInst = prices.shape
    print(f"Loaded price matrix: {T:,d} bars × {nInst} instruments\n")

    # parallel across CPU cores
    results: List[Tuple[int,int,float,float,float]] = Parallel(
        n_jobs=-1, backend="loky", prefer="processes")(
        delayed(eval_instrument)(i, prices[:, i]) for i in range(nInst))

    hdr = ("Inst", "Best-N", "Best-W", "Dyn-PnL", "StaticThr", "Static-PnL")
    print(f"{hdr[0]:>5}  {hdr[1]:>7}  {hdr[2]:>7}  {hdr[3]:>12}  "
          f"{hdr[4]:>10}  {hdr[5]:>12}")
    print("─"*72)

    tot_dyn = tot_static = 0.0
    for i, (N_opt, W_opt, pnl_dyn, thr_stat, pnl_stat) in enumerate(results):
        tot_dyn    += pnl_dyn
        tot_static += pnl_stat
        print(f"{i:5d}  {N_opt:7d}  {W_opt:7d}  {pnl_dyn:12,.2f}  "
              f"{thr_stat:10.4f}  {pnl_stat:12,.2f}")

    print("─"*72)
    print(f"Σ Net PnL (dynamic): {tot_dyn:,.2f}")
    print(f"Σ Net PnL (static) : {tot_static:,.2f}")


if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
pip_package.py (fixed)
──────────────────────
• Dynamic rolling-window PIP trader (no look-ahead).
• Back-test harness that writes KPI CSV & PNG equity plot.
• No hard-wired assumption about instrument count.
"""
from pathlib import Path
from typing  import List, Optional, Tuple, Dict, Callable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ═════════════ infrastructure stubs ═════════════
def export(fn: Callable) -> Callable:
    fn._is_exported = True
    return fn


class Trader:
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        raise NotImplementedError


# ═════════════ strategy hyper-params (grid-search output) ═════════════
BEST_N = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

FALLBACK_N, FALLBACK_W = 200, 25        # if price set has >50 instruments
POSITION_LIMIT         = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID               = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)
COMM_RATE              = 0.0005

# ═════════════ helpers ═════════════
def _causal_update(px: float,
                   last_extreme: float,
                   direction: int,
                   thr: float) -> Tuple[int, float]:
    if direction == 0:
        move = (px - last_extreme) / last_extreme if last_extreme else 0.0
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme

    if direction == 1:
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme

    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def _best_threshold(window: np.ndarray) -> float:
    if len(window) < 10:        # window too small → fallback
        return 0.05
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = _causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pos = dir_vec.astype(np.int32) * POSITION_LIMIT
        pnl = (pos[:-1] * np.diff(window)).sum()
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ═════════════ strategy class ═════════════
class CausalPIPTrader(Trader):

    class _State:
        __slots__ = ("n","w","next_cal","thr","dir","le")
        def __init__(self, n:int, w:int):
            self.n, self.w    = n, w
            self.next_cal     = n
            self.thr          = None
            self.dir          = 0
            self.le           = None

    def __init__(self, max_instruments: int | None = None):
        super().__init__()
        self._states: List[CausalPIPTrader._State] = []
        self._max_inst = max_instruments
        print("CausalPIPTrader initialised.")

    def _ensure_states(self, n_inst: int):
        """Expand state list if price file has more instruments than grid."""
        while len(self._states) < n_inst:
            idx = len(self._states)
            if idx < len(BEST_N):
                n, w = BEST_N[idx], BEST_W[idx]
            else:                          # fallback for overflow symbols
                n, w = FALLBACK_N, FALLBACK_W
            self._states.append(self._State(n, w))

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        n_inst, t_seen = prcSoFar.shape
        if self._max_inst and n_inst > self._max_inst:
            raise ValueError("Instrument count exceeded limit.")
        self._ensure_states(n_inst)

        positions = np.zeros(n_inst, np.int32)

        for i in range(n_inst):
            st    = self._states[i]
            price = prcSoFar[i, -1]

            if st.le is None:       # init last extreme
                st.le = price

            if t_seen >= st.next_cal:
                win_start = max(0, t_seen - st.n)
                st.thr    = _best_threshold(prcSoFar[i, win_start:t_seen])
                st.next_cal += st.w
                print(f"[t={t_seen:4d}] Inst {i:02d} new thr={st.thr:.4f}")

            if st.thr is None:
                continue

            st.dir, st.le = _causal_update(price, st.le, st.dir, st.thr)
            positions[i]  = st.dir * POSITION_LIMIT

        return positions


# ═════════════ back-test harness ═════════════
def run_backtest(prices_path: str = "prices.txt") -> pd.DataFrame:
    prices = np.loadtxt(Path(prices_path), delimiter=None)
    T, nI  = prices.shape
    tr     = CausalPIPTrader()
    rec = {"inst":[], "bar":[], "price":[], "pos":[], "pnl":[], "equity":[]}
    pos_prev = np.zeros(nI, np.int32)
    equity   = np.zeros(nI, np.float64)

    for t in range(1, T):
        pos = tr.Alg(prices[:t+1, :].T)   
        delta = pos - pos_prev
        pnl   = pos_prev * (prices[t] - prices[t-1])
        comm  = COMM_RATE * np.abs(delta) * prices[t]
        equity += pnl - comm

        for i in range(nI):
            rec["inst"].append(i)
            rec["bar"].append(t)
            rec["price"].append(prices[t, i])
            rec["pos"].append(pos[i])
            rec["pnl"].append(pnl[i]-comm[i])
            rec["equity"].append(equity[i])

        pos_prev = pos.copy()

    return pd.DataFrame(rec)


def _kpis(df: pd.DataFrame) -> pd.DataFrame:
    def agg(sub):
        tot = sub.pnl.sum()
        vol = sub.pnl.std(ddof=0)
        sharpe = tot / vol * np.sqrt(252*6.5*60) if vol else 0
        dd = (sub.equity.cummax() - sub.equity).max()
        return pd.Series({"TotalPnL": tot, "Sharpe": sharpe, "MaxDD": dd})
    return df.groupby("inst").apply(agg).reset_index()


def _plot_bottom(df: pd.DataFrame, kpi: pd.DataFrame, out: str):
    worst = kpi.nsmallest(max(1, len(kpi)//4), "Sharpe").inst
    plt.figure(figsize=(12,6))
    for i in worst:
        sub = df[df.inst == i]
        plt.plot(sub.bar, sub.equity, label=f"Inst {i:02d}", alpha=.6)
    plt.title("Bottom quartile equity curves")
    plt.legend(ncol=4, fontsize=7)
    plt.tight_layout()
    plt.savefig(out, dpi=150)
    plt.close()


# ═════════════ CLI run ═════════════
if __name__ == "__main__":
    OUT_CSV = "pip_kpis.csv"
    OUT_PNG = "pip_equity.png"

    df  = run_backtest("prices.txt")
    kpi = _kpis(df)
    kpi.to_csv(OUT_CSV, index=False)
    _plot_bottom(df, kpi, OUT_PNG)

    print(kpi.head(10).to_string(index=False))
    print(f"\nSaved KPI table → {OUT_CSV}")
    print(f"Saved equity plot → {OUT_PNG}")


In [ ]:
#!/usr/bin/env python3
"""
model_insights.py
─────────────────
Runs the dynamic PIP trader, collects per‐bar metrics and performs
statistical tests to pinpoint when and why it underperforms.

Metrics & tests:
  • Rolling volatility vs. PnL correlation
  • Rolling 5‐bar momentum vs. PnL correlation
  • Swing‐length (bars since last pivot) vs. PnL correlation
  • Group‐by quartile stats for each indicator
Only prints summary tables & correlations.
"""
import numpy as np
import pandas as pd
from pathlib import Path

def run_and_inspect(price_file="prices.txt"):
    prices = np.loadtxt(Path(price_file), delimiter=None)  # (T × nInst)
    T, nInst = prices.shape

    trader = CausalPIPTrader()
    # prepare records
    rec = {
        "inst": [], "bar": [], "price": [], "ret1": [], "vol20": [],
        "mom5": [], "thr": [], "dir": [], "swing_len": [],
        "pos": [], "pnl": []
    }
    pos_prev = np.zeros(nInst, dtype=int)
    last_dir = np.zeros(nInst, dtype=int)
    swing_len = np.zeros(nInst, dtype=int)

    # buffers for indicators
    ret_buf = np.full((20, nInst), np.nan)
    for t in range(1, T):
        # price return
        ret1 = (prices[t] - prices[t-1]) / prices[t-1]
        # update vol buffer
        ret_buf[t % 20] = ret1
        vol20 = np.nanstd(ret_buf, axis=0)
        # momentum
        if t >= 5:
            mom5 = (prices[t] - prices[t-5]) / prices[t-5]
        else:
            mom5 = np.zeros(nInst)

        # get positions
        pos = trader.Alg(prices[:t+1, :].T)
        pnl = pos_prev * (prices[t] - prices[t-1])

        # record per-inst metrics
        for i in range(nInst):
            rec["inst"].append(i)
            rec["bar"].append(t)
            rec["price"].append(prices[t, i])
            rec["ret1"].append(ret1[i])
            rec["vol20"].append(vol20[i])
            rec["mom5"].append(mom5[i])
            # threshold & direction from trader state
            st = trader._states[i]
            rec["thr"].append(st.thr if st.thr is not None else np.nan)
            rec["dir"].append(st.dir)
            # swing length: reset if dir changed
            if st.dir == last_dir[i]:
                swing_len[i] += 1
            else:
                swing_len[i] = 1
            last_dir[i] = st.dir
            rec["swing_len"].append(swing_len[i])
            rec["pos"].append(pos[i])
            rec["pnl"].append(pnl[i])

        pos_prev = pos.copy()

    df = pd.DataFrame(rec)

    # compute correlations
    corr_vol = df["vol20"].corr(df["pnl"])
    corr_mom = df["mom5"].corr(df["pnl"])
    corr_swing = df["swing_len"].corr(df["pnl"])
    corr_thr = df["thr"].corr(df["pnl"])

    print("Correlation of PnL with indicators:")
    print(f"  Rolling 20-bar vol  : {corr_vol:.4f}")
    print(f"  5-bar momentum      : {corr_mom:.4f}")
    print(f"  Swing length        : {corr_swing:.4f}")
    print(f"  Threshold (thr)     : {corr_thr:.4f}\n")

    # group-by quartiles
    for col in ["vol20", "mom5", "swing_len", "thr"]:
        df[f"{col}_q"] = pd.qcut(df[col].fillna(0), 4, labels=False, duplicates="drop")
        grp = df.groupby(f"{col}_q")["pnl"].agg(["mean","sum","count"])
        print(f"PnL by {col} quartile:")
        print(grp, "\n")

if __name__ == "__main__":
    run_and_inspect()


In [ ]:
##

In [ ]:
ROLL = 200           # bars
MIN_TR = 20          # minimum trades in window
SHARPE_FLOOR = -2
CONSEC = 3           # consecutive breaches
DD_PCT = 0.20        # 20 % of peak equity

bad = set()

for i in range(df.inst.nunique()):
    sub = df[df.inst == i].copy()

    # normalised pnl (return on notional)
    notional = sub["pos"].abs().shift().replace(0,np.nan) * sub["price"]
    ret = sub["pnl"] / notional
    roll_mu = ret.rolling(ROLL, min_periods=ROLL).mean()
    roll_sd = ret.rolling(ROLL, min_periods=ROLL).std(ddof=0)
    roll_sh = roll_mu / roll_sd * np.sqrt(252*6.5*60)

    # trades in window
    trade_ct = (sub["pos"].diff().abs() > 0).astype(int).rolling(ROLL).sum()

    breach = ((roll_sh < SHARPE_FLOOR) & (trade_ct >= MIN_TR)).astype(int)
    if breach.rolling(CONSEC).sum().max() >= CONSEC:
        bad.add(i)

    # percentage draw-down
    peak = sub["equity"].cummax()
    dd_pct = (peak - sub["equity"]) / peak.replace(0,np.nan)
    if (dd_pct > DD_PCT).any():
        bad.add(i)


In [ ]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
from pathlib import Path

# ────────────────────────── helpers ────────────────────────── #
def load_prices(prices_file="prices.txt"):
    """Load (timesteps × instruments) price matrix."""
    return np.loadtxt(Path(prices_file), delimiter=None)

def causal_signal(series: np.ndarray, threshold: float) -> np.ndarray:
    """
    Fully-causal zig-zag direction: +1 long / –1 short / 0 flat.
    """
    n   = len(series)
    pos = np.zeros(n, dtype=int)
    last_extreme = series[0]
    direction    = None

    for i in range(1, n):
        move = (series[i] - last_extreme) / last_extreme
        if direction is None:
            if abs(move) >= threshold:
                direction    = 'up' if move > 0 else 'down'
                last_extreme = series[i]
        else:
            if direction == 'up':
                if series[i] > last_extreme:
                    last_extreme = series[i]
                else:
                    rev = (last_extreme - series[i]) / last_extreme
                    if rev >= threshold:
                        direction    = 'down'
                        last_extreme = series[i]
            else:                                 # direction == 'down'
                if series[i] < last_extreme:
                    last_extreme = series[i]
                else:
                    rev = (series[i] - last_extreme) / last_extreme
                    if rev >= threshold:
                        direction    = 'up'
                        last_extreme = series[i]
        pos[i] = 1 if direction=='up' else (-1 if direction=='down' else 0)
    return pos

def trade_returns(series: np.ndarray, pos: np.ndarray) -> list:
    """
    Extract contiguous +1 / –1 runs and compute signed % returns for each.
    """
    trades = []
    cur_pos   = pos[0]
    entry_idx = 0 if cur_pos else None
    for i in range(1, len(series)):
        if pos[i] != cur_pos:
            if cur_pos:
                pnl = (series[i-1] - series[entry_idx]) / series[entry_idx] * cur_pos
                trades.append(pnl)
            cur_pos   = pos[i]
            entry_idx = i if cur_pos else None
    if cur_pos and entry_idx is not None:
        pnl = (series[-1] - series[entry_idx]) / series[entry_idx] * cur_pos
        trades.append(pnl)
    return trades

def bar_winrate(series: np.ndarray, pos: np.ndarray) -> float:
    """
    For each bar where pos != 0, check if next-bar return has same sign.
    """
    correct = total = 0
    for i in range(len(series)-1):          # last bar has no next return
        if pos[i] != 0:
            ret = series[i+1] - series[i]
            total += 1
            if ret * pos[i] > 0:
                correct += 1
    return correct / total if total else np.nan

# ──────────────────────── main analysis ────────────────────── #
if __name__ == "__main__":
    try:
        prices = load_prices("prices.txt")
        print("Loaded prices.txt\n")
    except FileNotFoundError:
        np.random.seed(0)
        prices = np.cumsum(np.random.randn(1000, 50), axis=0) + 100.0
        print("Using synthetic data (1000 × 50)\n")

    T, n_inst = prices.shape
    threshold = 0.01   # 1 % pivot threshold

    rows = []
    for inst in range(n_inst):
        series = prices[:, inst]
        pos    = causal_signal(series, threshold)

        # trade-level stats
        trades = trade_returns(series, pos)
        num_trades = len(trades)
        total_pnl  = float(np.sum(trades))          if num_trades else 0.0
        mean_pnl   = float(np.mean(trades))         if num_trades else 0.0
        std_pnl    = float(np.std(trades, ddof=0))  if num_trades else 0.0
        wins       = sum(r > 0 for r in trades)
        losses     = sum(r < 0 for r in trades)
        win_rate   = wins / num_trades if num_trades else np.nan

        # consecutive streaks
        runs_w = runs_l = []
        if num_trades:
            runs_w = []; runs_l = []
            cw = cl = 0
            for r in trades:
                if r > 0:
                    cw += 1
                    if cl:
                        runs_l.append(cl); cl = 0
                elif r < 0:
                    cl += 1
                    if cw:
                        runs_w.append(cw); cw = 0
            if cw: runs_w.append(cw)
            if cl: runs_l.append(cl)
        max_w = max(runs_w) if runs_w else 0
        max_l = max(runs_l) if runs_l else 0
        avg_w = float(np.mean(runs_w)) if runs_w else 0.0
        avg_l = float(np.mean(runs_l)) if runs_l else 0.0

        # bar-by-bar directional accuracy
        sig_winrate = bar_winrate(series, pos)

        rows.append({
            "Instrument":        inst,
            "Trades":            num_trades,
            "Total PnL":         total_pnl,
            "Mean PnL":          mean_pnl,
            "PnL Std":           std_pnl,
            "Wins":              wins,
            "Losses":            losses,
            "Win Rate":          win_rate,
            "Signal WinRate":    sig_winrate,
            "Max Cons Wins":     max_w,
            "Max Cons Losses":   max_l,
            "Avg Cons Wins":     avg_w,
            "Avg Cons Losses":   avg_l,
        })

    df = pd.DataFrame(rows)
    pd.set_option("display.float_format", "{:.6f}".format)
    print(df.to_string(index=False))


In [ ]:
#!/usr/bin/env python3
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def load_prices(prices_file="prices.txt"):
    """
    Load (timesteps × instruments) price matrix.
    Assumes whitespace-delimited, oldest row first.
    """
    return np.loadtxt(Path(prices_file), delimiter=None)

def find_pips(prices: np.ndarray, threshold: float):
    """
    Zig-zag pivot finder, causal.
    Returns list of (idx, price).
    """
    n = len(prices)
    pips = [(0, prices[0])]
    last_extreme = prices[0]
    direction = None

    for i in range(1, n):
        move = (prices[i] - last_extreme) / last_extreme

        if direction is None:
            if abs(move) >= threshold:
                direction = 'up' if move > 0 else 'down'
                last_extreme = prices[i]
                pips.append((i, prices[i]))
        else:
            if direction=='up' and prices[i] > last_extreme:
                last_extreme = prices[i]
            elif direction=='down' and prices[i] < last_extreme:
                last_extreme = prices[i]

            rev = ((last_extreme - prices[i]) / last_extreme
                   if direction=='up'
                   else (prices[i] - last_extreme) / last_extreme)
            if rev >= threshold:
                pips.append((i, last_extreme))
                direction = 'down' if direction=='up' else 'up'
                last_extreme = prices[i]

    if pips[-1][0] != n-1:
        pips.append((n-1, prices[-1]))
    return pips

def causal_signal(series: np.ndarray, threshold: float):
    """
    One-pass, fully causal up/down signal:
    +1 long, -1 short, 0 flat until first swing.
    """
    n = len(series)
    pos = np.zeros(n, dtype=int)
    last_extreme = series[0]
    direction = None

    for i in range(1, n):
        move = (series[i] - last_extreme) / last_extreme

        if direction is None:
            if abs(move) >= threshold:
                direction = 'up' if move > 0 else 'down'
                last_extreme = series[i]
        else:
            if direction=='up':
                if series[i] > last_extreme:
                    last_extreme = series[i]
                else:
                    rev = (last_extreme - series[i]) / last_extreme
                    if rev >= threshold:
                        direction = 'down'
                        last_extreme = series[i]
            else:
                if series[i] < last_extreme:
                    last_extreme = series[i]
                else:
                    rev = (series[i] - last_extreme) / last_extreme
                    if rev >= threshold:
                        direction = 'up'
                        last_extreme = series[i]

        pos[i] = 1 if direction=='up' else (-1 if direction=='down' else 0)
    return pos

# ───── Example Usage ─────
prices   = load_prices("prices.txt")      # shape (1000, 50)
instr    = 3                              # instrument index
t1, t2   = 850, 900                       # window
threshold = 0.01                          # 1% pip threshold

series = prices[t1:t2+1, instr]
times  = np.arange(t1, t2+1)

pos    = causal_signal(series, threshold)
pivots = find_pips(series, threshold)
idxs, vals = zip(*pivots)

# ───── Plot ─────
fig, ax1 = plt.subplots(figsize=(12,5))

# shade under price
ax1.fill_between(times, series, series.min(),
                 where=pos>0, facecolor='green', alpha=0.2,
                 step='post')
ax1.fill_between(times, series, series.min(),
                 where=pos<0, facecolor='red',   alpha=0.2,
                 step='post')

# price and pivots
ax1.plot(times, series, color='black', linewidth=1, label=f"Inst {instr}")
ax1.scatter(times[list(idxs)], vals,
            color='blue', s=40, label='PIPs')

ax1.set_xlabel("Timestep")
ax1.set_ylabel("Price")
ax1.set_title("Fully Causal PIP-Based Up/Down Indicator")

# ─── overlay actual position predictions ──────────────
ax2 = ax1.twinx()
ax2.step(times, pos, where='post',
         color='gray', linewidth=1, label='Position')
ax2.set_ylim(-1.2, 1.2)
ax2.set_ylabel("Position (+1 long / -1 short)")

# combine legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines+lines2, labels+labels2,
           loc='upper left', fontsize='small')

fig.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def load_prices(prices_file="prices.txt"):
    """
    Load (timesteps × instruments) price matrix.
    Assumes whitespace-delimited, oldest row first.
    """
    return np.loadtxt(Path(prices_file), delimiter=None)

def find_pips(prices: np.ndarray, threshold: float):
    """
    (Optional) Zig-zag pivot finder, causal:
    you only ever use prices up to the current bar to detect pivots.
    Returns list of (idx, price).
    """
    n = len(prices)
    pips = [(0, prices[0])]
    last_extreme = prices[0]
    direction = None

    for i in range(1, n):
        move = (prices[i] - last_extreme) / last_extreme

        if direction is None:
            if abs(move) >= threshold:
                direction = 'up' if move > 0 else 'down'
                last_extreme = prices[i]
                pips.append((i, prices[i]))
        else:
            # extend swing
            if direction=='up' and prices[i] > last_extreme:
                last_extreme = prices[i]
            elif direction=='down' and prices[i] < last_extreme:
                last_extreme = prices[i]
            # check for reversal
            rev = ((last_extreme - prices[i]) / last_extreme if direction=='up'
                   else (prices[i] - last_extreme) / last_extreme)
            if rev >= threshold:
                pips.append((i, last_extreme))
                direction = 'down' if direction=='up' else 'up'
                last_extreme = prices[i]
    if pips[-1][0] != n-1:
        pips.append((n-1, prices[-1]))
    return pips

def causal_signal(series: np.ndarray, threshold: float):
    """
    One-pass, fully causal up/down signal:
    +1 (long) when current swing is up,
    –1 (short) when it's down,
    0 until first swing confirms.
    """
    n = len(series)
    pos = np.zeros(n, dtype=int)
    last_extreme = series[0]
    direction = None

    for i in range(1, n):
        move = (series[i] - last_extreme) / last_extreme

        if direction is None:
            # wait for first threshold breach
            if abs(move) >= threshold:
                direction = 'up' if move > 0 else 'down'
                last_extreme = series[i]
        else:
            # extend current swing
            if direction=='up' and series[i] > last_extreme:
                last_extreme = series[i]
            elif direction=='down' and series[i] < last_extreme:
                last_extreme = series[i]
            # check for reversal
            rev = ((last_extreme - series[i]) / last_extreme if direction=='up'
                   else (series[i] - last_extreme) / last_extreme)
            if rev >= threshold:
                direction = 'down' if direction=='up' else 'up'
                last_extreme = series[i]

        # set position at this bar
        pos[i] = 1 if direction=='up' else (-1 if direction=='down' else 0)

    return pos

# ───── Example Usage ─────
prices   = load_prices("prices.txt")      # shape (1000, 50)
instr    = 3                             # pick instrument 7
t1, t2 =  850, 900                      # window
threshold = 0.01                      # 0.1% pip threshold

series = prices[:, instr][t1:t2+1]
times  = np.arange(t1, t2+1)

# get fully causal pos signal
pos = causal_signal(series, threshold)

# optional pivot markers (also causal)
pivots = find_pips(series, threshold)
idxs, vals = zip(*pivots)

# ───── Plot ─────
plt.figure(figsize=(12, 5))

# background shading: green when pos>0, red when pos<0
plt.fill_between(times, series.min(), series.max(),
                 where=pos>0, facecolor='green', alpha=0.2, step='post')
plt.fill_between(times, series.min(), series.max(),
                 where=pos<0, facecolor='red',   alpha=0.2, step='post')

# price line + pivots
plt.plot(times, series, color='black', linewidth=1, label=f"Inst {instr}")
plt.scatter(times[list(idxs)], vals, color='blue', s=40, label='PIPs')

# thin black lines at every timestep
for t in times:
    plt.axvline(t, color='black', linewidth=0.3, alpha=0.5)

plt.title("Fully Causal PIP-Based Up/Down Indicator")
plt.xlabel("Timestep")
plt.ylabel("Price") 
plt.legend(loc='upper left', fontsize='small')
plt.tight_layout()
plt.show()


In [ ]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
from pathlib import Path

def load_prices(prices_file="prices.txt"):
    """
    Load (timesteps × instruments) price matrix.
    Assumes whitespace-delimited, oldest row first.
    """
    return np.loadtxt(Path(prices_file), delimiter=None)

def causal_signal(series: np.ndarray, threshold: float):
    """
    One-pass, fully causal up/down signal:
    +1 (long) when current swing is up,
    -1 (short) when it's down,
    0 until first swing confirms.
    """
    n = len(series)
    pos = np.zeros(n, dtype=int)
    last_extreme = series[0]
    direction = None

    for i in range(1, n):
        move = (series[i] - last_extreme) / last_extreme
        if direction is None:
            if abs(move) >= threshold:
                direction = 'up' if move > 0 else 'down'
                last_extreme = series[i]
        else:
            if direction == 'up':
                if series[i] > last_extreme:
                    last_extreme = series[i]
                else:
                    rev = (last_extreme - series[i]) / last_extreme
                    if rev >= threshold:
                        direction = 'down'
                        last_extreme = series[i]
            else:
                if series[i] < last_extreme:
                    last_extreme = series[i]
                else:
                    rev = (series[i] - last_extreme) / last_extreme
                    if rev >= threshold:
                        direction = 'up'
                        last_extreme = series[i]
        pos[i] = 1 if direction == 'up' else (-1 if direction == 'down' else 0)
    return pos

def main():
    # Load prices, with synthetic fallback
    try:
        prices = load_prices("prices.txt")
        print("Loaded prices.txt")
    except FileNotFoundError:
        np.random.seed(0)
        prices = np.cumsum(np.random.randn(1000, 50), axis=0) + 100.0
        print("Using synthetic data (1000×50)")

    T, n_inst = prices.shape
    threshold = 0.01  # pivot threshold

    records = []
    for instr in range(n_inst):
        series = prices[:, instr]
        pos = causal_signal(series, threshold)
        n = len(series)

        # identify trades as contiguous runs of non-zero pos
        trades = []
        last_p = pos[0]
        entry = 0 if last_p != 0 else None
        for i in range(1, n):
            if pos[i] != last_p:
                # close previous trade
                if last_p != 0:
                    trades.append((entry, i-1, last_p))
                # start new if non-zero
                entry = i if pos[i] != 0 else None
                last_p = pos[i]
        # end-of-series close
        if last_p != 0:
            trades.append((entry, n-1, last_p))

        # compute trade returns
        trade_returns = []
        for (e, x, p) in trades:
            ret = (series[x] - series[e]) / series[e] * p
            trade_returns.append(ret)

        num_trades = len(trade_returns)
        total_pnl = np.sum(trade_returns) if num_trades > 0 else 0.0
        mean_pnl  = np.mean(trade_returns) if num_trades > 0 else 0.0
        std_pnl   = np.std(trade_returns, ddof=0) if num_trades > 0 else 0.0
        wins      = sum(1 for r in trade_returns if r > 0)
        losses    = sum(1 for r in trade_returns if r < 0)
        win_rate  = wins / num_trades if num_trades > 0 else np.nan

        # consecutive trade runs
        runs_win = []
        runs_loss = []
        cw = cl = 0
        for r in trade_returns:
            if r > 0:
                cw += 1
                if cl > 0:
                    runs_loss.append(cl)
                    cl = 0
            elif r < 0:
                cl += 1
                if cw > 0:
                    runs_win.append(cw)
                    cw = 0
        if cw > 0:
            runs_win.append(cw)
        if cl > 0:
            runs_loss.append(cl)

        max_cons_wins   = max(runs_win) if runs_win else 0
        max_cons_losses = max(runs_loss) if runs_loss else 0
        avg_cons_wins   = np.mean(runs_win) if runs_win else 0.0
        avg_cons_losses = np.mean(runs_loss) if runs_loss else 0.0

        records.append({
            "Instrument": instr,
            "Num Trades": num_trades,
            "Total PnL": total_pnl,
            "Mean PnL": mean_pnl,
            "PnL Std": std_pnl,
            "Win Rate": win_rate,
            "Wins": wins,
            "Losses": losses,
            "Max Cons Wins": max_cons_wins,
            "Max Cons Losses": max_cons_losses,
            "Avg Cons Wins": avg_cons_wins,
            "Avg Cons Losses": avg_cons_losses
        })

    df = pd.DataFrame(records)
    print("\nTrade-Level Metrics per Instrument:\n")
    print(df.to_string(index=False, float_format="{:.6f}".format))

if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
per_instrument_gridsearch.py

Grid-search N and W **per instrument** for the dynamic PIP trader,
accounting for commission on dollar turnover.  Prints progress and
a final summary table with best (N, W), total PnL and PnL std for
each of the 50 instruments.
"""

import numpy as np
import pandas as pd
from itertools import product
from pathlib import Path

# ────────────────────────── hyper-parameters ──────────────────────────── #
N_VALUES         = [100, 200, 300, 400]
W_VALUES         = [50, 100, 200]
COMMISSION_RATE  = 0.000     # 0.05% per dollar traded
POSITION_LIMIT   = 10_000.0
THR_GRID         = np.arange(0.0, 0.700+0.0005, 0.0005)

# ───────────────────────── helper functions ────────────────────────────── #
def causal_update(px: float, le: float, d: int, thr: float):
    if d == 0:
        mv = (px - le) / le
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, le
    if d == 1:
        if px > le:
            return 1, px
        if (le - px) / le >= thr:
            return -1, px
        return 1, le
    # d == -1
    if px < le:
        return -1, px
    if (px - le) / le >= thr:
        return 1, px
    return -1, le

def best_threshold(window: np.ndarray, thr_grid: np.ndarray, position_limit: float) -> float:
    """
    Find threshold that maximizes PnL over `window` using
    the causal_update logic and given position_limit.
    """
    best_thr = thr_grid[0]
    best_pnl = -np.inf
    for thr in thr_grid:
        d, le = 0, window[0]
        pnl = 0.0
        for p0, p1 in zip(window[:-1], window[1:]):
            d, le = causal_update(p1, le, d, thr)
            pnl += d * position_limit * (p1 - p0)
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

def backtest_instrument(series: np.ndarray,
                        N: int,
                        W: int,
                        commission_rate: float,
                        thr_grid: np.ndarray,
                        position_limit: float) -> np.ndarray:
    """
    Back-test one instrument price `series` (length T).
    Re-calibrate threshold every W bars using last Nbars.
    Returns net-PnL time series of length T-1 (commission deducted).
    """
    T = series.size
    pnl = np.zeros(T-1, dtype=float)
    # state
    next_cal = N
    thr      = None
    d        = 0
    le       = series[0]
    prev_pos = 0.0

    for t in range(1, T):
        px = series[t]
        # recalibrate threshold
        if t >= next_cal:
            window = series[max(0, t-N): t]
            thr = best_threshold(window, thr_grid, position_limit)
            next_cal += W

        # decide position
        if thr is not None:
            d, le = causal_update(px, le, d, thr)
        pos = d * position_limit

        # compute bar PnL
        price_diff = px - series[t-1]
        trade_pnl  = prev_pos * price_diff

        # commission on dollar turnover
        turnover   = abs(pos - prev_pos) * px
        commission = commission_rate * turnover

        pnl[t-1]   = trade_pnl - commission
        prev_pos   = pos

    return pnl

# ──────────────────────── main grid-search ───────────────────────────── #
if __name__ == "__main__":
    # load price data: shape (T,50) → transpose to (50,T)
    prices = np.loadtxt(Path("prices.txt"), delimiter=None).T

    results = []
    for inst in range(prices.shape[0]):
        series = prices[inst]
        best_pnl = -np.inf
        best_std = np.nan
        best_N   = None
        best_W   = None

        print(f"\n--- Instrument {inst:02d} ---")
        for N, W in product(N_VALUES, W_VALUES):
            print(f" Testing N={N:3d}, W={W:3d} ...", end="", flush=True)
            pnl_series = backtest_instrument(
                series,
                N,
                W,
                COMMISSION_RATE,
                THR_GRID,
                POSITION_LIMIT
            )
            total_pnl = pnl_series.sum()
            std_pnl   = pnl_series.std(ddof=0)
            print(f" PnL={total_pnl:.2f}, Std={std_pnl:.2f}")

            if total_pnl > best_pnl:
                best_pnl = total_pnl
                best_std = std_pnl
                best_N   = N
                best_W   = W

        print(f" => Best: N={best_N}, W={best_W}, TotalPnL={best_pnl:.2f}, Std={best_std:.2f}")

        results.append({
            "Instrument": inst,
            "Best_N":      best_N,
            "Best_W":      best_W,
            "TotalPnL":    best_pnl,
            "PnL_Std":     best_std
        })

    df = pd.DataFrame(results)
    pd.set_option("display.float_format", "{:.2f}".format)
    print("\n=== Per-Instrument Grid-Search Results ===")
    print(df.to_string(index=False))
